In [1]:
import json
import uuid
from functools import lru_cache
from typing import Dict, List, Optional, Tuple

import chromadb
import faiss
import numpy as np
import pandas as pd
import torch
from chromadb.api.types import Documents, Embeddings
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import EmbeddingFunction
from faiss import IndexFlatIP
from sklearn.preprocessing import LabelEncoder
from torch import nn
from tqdm import tqdm
from transformers import (
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

# Инициализация устройства один раз
if torch.cuda.is_available():
    device = torch.device("cuda")
    # Оптимизация для Tensor Cores (если доступно)
    torch.backends.cuda.matmul.allow_tf32 = True  
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

In [2]:
class Classifier:
    """Класс для классификации текстов и получения эмбеддингов"""
    
    def __init__(self, 
                 tokenizer=None, 
                 model=None, 
                 mapping_dict: Dict[int, str] = None,
                 device: str = None):
        """
        Инициализация классификатора
        
        Args:
            tokenizer: токенизатор (если None, загружается модель по умолчанию)
            model: модель классификации
            mapping_dict: словарь соответствия id классов и их названий
            device: устройство для вычислений (cuda/cpu)
        """
        self.tokenizer = tokenizer or AutoTokenizer.from_pretrained("intfloat/multilingual-e5-base")
        self.model = model or AutoModelForSequenceClassification.from_pretrained(
            './results/checkpoint-30500', 
            num_labels=len(mapping_dict) if mapping_dict else None
        )
        self.mapping_dict = mapping_dict or {}
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.model.eval()
    
    def get_embeddings(self, texts: List[str]) -> torch.Tensor:
        """
        Получение векторных представлений для списка текстов
        
        Args:
            texts: список текстов для векторизации
            
        Returns:
            Тензор с эмбеддингами текстов
        """
        inputs = self.tokenizer(
            texts, 
            padding=True, 
            truncation=True, 
            return_tensors="pt"
        ).to(self.device)
        
        with torch.no_grad():
            last_hidden_state = self.model(**inputs, output_hidden_states=True).hidden_states[-1]
            return last_hidden_state[:, 0, :].contiguous()
    
    def predict(self, text: str, top_k: int = 3) -> Tuple[torch.Tensor, List[str]]:
        """
        Предсказание классов для текста
        
        Args:
            text: входной текст
            top_k: количество возвращаемых топовых классов
            
        Returns:
            Кортеж (вероятности классов, названия классов)
        """
        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
        with torch.no_grad():
            logits = self.model(**inputs).logits
            probs = torch.nn.functional.softmax(logits, dim=1).cpu()[0]
        
        top_probs, top_indices = torch.topk(probs, k=top_k)
        predicted_classes = [self.mapping_dict[idx.item()] for idx in top_indices]
        return top_probs, predicted_classes


class VectorDB:
    """Класс для работы с векторной базой данных ChromaDB"""
    
    class E5EmbeddingFunction(EmbeddingFunction):
        """Функция для создания эмбеддингов с использованием классификатора"""
        
        def __init__(self, classifier: Classifier):
            self.classifier = classifier
        
        def __call__(self, input: List[str]) -> List[List[float]]:
            """Конвертация текстов в эмбеддинги"""
            embeddings = self.classifier.get_embeddings(input)
            return embeddings.detach().cpu().numpy().tolist()
    
    def __init__(self, classifier: Classifier, db_path: str = "./chroma_db"):
        """
        Инициализация векторной базы данных
        
        Args:
            classifier: экземпляр Classifier
            db_path: путь к хранилищу ChromaDB
        """
        self.client = chromadb.PersistentClient(path=db_path)
        self.collection = self.client.create_collection(
            name="products",
            embedding_function=self.E5EmbeddingFunction(classifier),
            metadata={"hnsw:space": "cosine"}
        )
    
    def add_documents(self, data: pd.DataFrame):
        """Добавление документов в коллекцию"""
        batch_size = 32
        for i in tqdm(range(0, len(data), batch_size), desc="Processing batches"):
            batch = data.iloc[i:i+batch_size]
            
            documents = batch["text"].tolist()
            metadatas = [
                {
                    "id": str(row['id']),
                    "label": str(row['label']),
                    "code": row['code']
                    # Убрали дублирование текста в metadata
                } for _, row in batch.iterrows()
            ]
            ids = [str(uuid.uuid4()) for _ in range(len(batch))]
            
            self.collection.add(
                documents=documents,
                metadatas=metadatas,
                ids=ids
            )
    
    def query(self, query_text: str, where_filter: Optional[Dict] = None, n_results: int = 10):
        """
        Поиск в векторной базе данных
        
        Args:
            query_text: поисковый запрос
            where_filter: фильтр по метаданным
            n_results: количество возвращаемых результатов
            
        Returns:
            Результаты поиска
        """
        return self.collection.query(
            query_texts=[query_text],
            where=where_filter,
            n_results=n_results
        )


class Reranker:
    def __init__(self, 
                 model_path: str = './results/checkpoint-30500',
                 base_model: str = "intfloat/multilingual-e5-base",
                 device: str = None,
                 max_length: int = 512,
                 cache_size: int = 10000):
        """
        Реранкер на основе вашей fine-tuned модели E5
        
        Args:
            model_path: путь к чекпоинту
            base_model: базовая модель токенизатора
            device: "cuda" или "cpu"
            max_length: максимальная длина текста
            cache_size: размер кэша эмбеддингов
        """
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(base_model)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path).to(self.device)
        self.max_length = max_length
        self.model.eval()
        self._cache = lru_cache(maxsize=cache_size)(self._compute_embedding)

    def _compute_embedding(self, text: str) -> np.ndarray:
        """Вычисление эмбеддинга без кэширования"""
        inputs = self.tokenizer(
            text,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
            # Берем эмбеддинг из последнего слоя [CLS] токена
            embedding = outputs.hidden_states[-1][:, 0, :].cpu().numpy()[0]
        
        return embedding / np.linalg.norm(embedding)  # Нормализация

    def rerank(self, 
               query: str, 
               documents: List[Dict], 
               top_k: Optional[int] = None,
               batch_size: int = 32) -> List[Dict]:
        """
        Реранжирование документов относительно запроса
        
        Args:
            query: поисковый запрос
            documents: список документов [{"text": "...", ...}]
            top_k: вернуть только top_k результатов
            batch_size: размер батча для обработки
            
        Returns:
            Отсортированные документы с добавленными полями score и embedding
        """
        if not documents:
            return []

        # Эмбеддинг запроса
        query_embedding = self._cache(query)
        
        # Обработка документов батчами
        doc_embeddings = []
        for i in tqdm(range(0, len(documents), batch_size)):
            batch = documents[i:i+batch_size]
            texts = [doc["text"] for doc in batch]
            
            # Получаем эмбеддинги для батча
            embeddings = [self._cache(text) for text in texts]
            doc_embeddings.extend(embeddings)
        
        # Вычисляем косинусную близость
        scores = np.dot(doc_embeddings, query_embedding)
        
        # Обновляем документы
        for doc, score, emb in zip(documents, scores, doc_embeddings):
            doc.update({
                "score": float(score),
            })
        
        # Сортировка по убыванию релевантности
        documents.sort(key=lambda x: x["score"], reverse=True)
        
        return documents[:top_k] if top_k else documents

    def precompute(self, documents: List[Dict]):
        """Предварительное вычисление эмбеддингов"""
        for doc in tqdm(documents, desc="Прекомпьютинг эмбеддингов"):
            self._cache(doc["text"])

class SearchPipeline:
    """Основной пайплайн для поиска с классификацией и реранжированием"""
    
    def __init__(self, 
                 classifier: Classifier = None,
                 vector_db: VectorDB = None,
                 reranker: Reranker = None,
                 threshold: float = 0.7,
                 mapping_dict: Dict[int, str] = None,
                 semantic_weight: float = 0.6):
        """
        Инициализация пайплайна
        
        Args:
            semantic_weight: вес семантического поиска (0.6 = 60% влияния)
        """
        self.classifier = classifier or Classifier(mapping_dict=mapping_dict)
        self.vector_db = vector_db or VectorDB(self.classifier)
        self.reranker = reranker or Reranker()
        self.threshold = threshold
        self.semantic_weight = semantic_weight
    
    def _format_results(self, results) -> List[Dict]:
        """Форматирование результатов из ChromaDB с удалением дублей по text"""
        seen_texts = set()
        formatted = []
        
        for doc, meta, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
            text = doc.strip()  # Нормализация текста
            if text not in seen_texts:
                formatted.append({
                    "text": text,
                    "class": meta["label"],
                    "score": float(1 - dist),
                    "distance": float(dist),
                    "metadata": meta,
                    "id": meta["id"]  # Сохраняем оригинальный id
                })
                seen_texts.add(text)
        return formatted
    
    def _fuse_results(self, semantic_res, class_res):
        """Объединение результатов с удалением дублей по text"""
        fused = []
        seen_texts = set()
        
        # Нормализация scores
        sem_scores = [x['score'] for x in semantic_res] or [1]
        class_scores = [x['score'] for x in class_res] or [1]
        
        max_sem = max(sem_scores)
        max_class = max(class_scores)
        
        # Создаем словарь для быстрого доступа по text
        class_results_dict = {doc['text'].strip(): doc for doc in class_res}
        
        # Обрабатываем семантические результаты
        for doc in semantic_res:
            text = doc['text'].strip()
            if text in class_results_dict:
                # Если документ есть в обоих результатах - объединяем
                class_doc = class_results_dict[text]
                combined_score = (self.semantic_weight * (doc['score'] / max_sem) + 
                                (1 - self.semantic_weight) * (class_doc['score'] / max_class))
                fused.append({
                    **doc,
                    "combined_score": combined_score,
                    "match_type": "both"
                })
            else:
                # Только семантический результат
                fused.append({
                    **doc,
                    "combined_score": self.semantic_weight * (doc['score'] / max_sem),
                    "match_type": "semantic"
                })
            seen_texts.add(text)
        
        # Добавляем класс-ориентированные результаты, которых нет в семантических
        for doc in class_res:
            text = doc['text'].strip()
            if text not in seen_texts:
                fused.append({
                    **doc,
                    "combined_score": (1 - self.semantic_weight) * (doc['score'] / max_class),
                    "match_type": "class"
                })
        
        # Сортируем по комбинированному score
        fused.sort(key=lambda x: x['combined_score'], reverse=True)
        return fused
    
    def search(self, query_text: str, top_k: int = 3) -> Dict:
        """
        Гибридный поиск с защитой от дубликатов по text
        
        Args:
            query_text: поисковый запрос
            top_k: количество возвращаемых результатов
            
        Returns:
            Словарь с результатами поиска
        """
        # 1. Векторизация запроса
        query_embedding = self.classifier.get_embeddings([query_text])[0]
        
        # 2. Параллельные запросы
        # 2.1. Семантический поиск (без фильтрации)
        semantic_results = self.vector_db.query(
            query_text=query_text,
            n_results=top_k*3
        )
        
        # 2.2. Класс-ориентированный поиск
        probs, classes = self.classifier.predict(query_text)
        if probs[0] >= self.threshold:
            class_filter = {"label": {"$eq": classes[0]}}
            strategy = "single_class"
            used_classes = [classes[0]]
        else:
            class_filter = {"$or": [{"label": c} for c in classes[:3]]}
            strategy = "multi_class"
            used_classes = classes[:3]
        
        class_results = self.vector_db.query(
            query_text=query_text,
            where_filter=class_filter,
            n_results=top_k*2
        )
        
        # 3. Объединение и реранжирование
        formatted_semantic = self._format_results(semantic_results)
        formatted_class = self._format_results(class_results)
        
        fused = self._fuse_results(formatted_semantic, formatted_class)
        
        # Фильтрация дублей перед реранжированием
        unique_texts = set()
        deduplicated = []
        for doc in fused:
            text = doc['text'].strip()
            if text not in unique_texts:
                deduplicated.append(doc)
                unique_texts.add(text)
        
        reranked = self.reranker.rerank(query_text, deduplicated, top_k=top_k*2)
        
        # Финальная проверка дублей (на случай если reranker изменил тексты)
        final_results = []
        final_texts = set()
        for doc in reranked:
            text = doc['text'].strip()
            if text not in final_texts:
                final_results.append(doc)
                final_texts.add(text)
            if len(final_results) >= top_k:
                break
        
        return {
            "strategy": strategy,
            "used_classes": used_classes,
            "results": final_results[:top_k]
        }

In [3]:
# Пример использования
def load_data(data_path: str, id2label_path: str) -> Tuple[pd.DataFrame, Dict[int, str]]:
    """Загрузка данных и маппинга классов"""
    with open(id2label_path, 'r', encoding='utf-8') as f:
        id2label = {int(k): v for k, v in json.load(f).items()}
    
    df = pd.read_csv(data_path)
    df = df.drop(columns=["Отдел", "Группа", "Класс", "Вид", "Подвид"])\
           .drop_duplicates()\
           .reset_index(drop=True)
    
    label2id = {v: k for k, v in id2label.items()}
    df = df.assign(
        text=df['Наименование с характеристикой'],
        id=df['Ведомственный классификатор'].map(label2id),
        label=df['Ведомственный классификатор'],
        code=df['Код']
    ).loc[:, ['text', 'label', 'code', 'id']]
    
    return df, id2label

# Инициализация пайплайна
data, id2label = load_data('./data/ЕНСТРУ.csv', './data/id2label.json')

In [4]:
pipeline = SearchPipeline(mapping_dict=id2label)

# Добавление документов
pipeline.vector_db.add_documents(data)

Processing batches: 100%|██████████| 1104/1104 [01:35<00:00, 11.53it/s]


In [24]:
# Поиск
results = pipeline.search("Рейка Ω-образного типа", top_k=3)

100%|██████████| 1/1 [00:00<00:00, 19328.59it/s]


In [25]:
results['results']

[{'text': 'Рейка С-образного типа, монтажная',
  'class': '251123.600',
  'score': 0.6621107459068298,
  'distance': 0.33779627084732056,
  'metadata': {'label': '251123.600',
   'id': '850.0',
   'code': '251123.600.000040'},
  'id': '850.0',
  'combined_score': 1.0,
  'match_type': 'both'},
 {'text': 'Рейка G-образного типа, монтажная',
  'class': '251123.600',
  'score': 0.6606615781784058,
  'distance': 0.33930504322052,
  'metadata': {'id': '850.0',
   'label': '251123.600',
   'code': '251123.600.000041'},
  'id': '850.0',
  'combined_score': 0.9977215888301777,
  'match_type': 'both'},
 {'text': 'Рейка Ω-образного типа (омега тип), монтажная',
  'class': '251123.600',
  'score': 0.6294140815734863,
  'distance': 0.37079089879989624,
  'metadata': {'label': '251123.600',
   'id': '850.0',
   'code': '251123.600.000104'},
  'id': '850.0',
  'combined_score': 0.950174505971457,
  'match_type': 'both'}]

## Benchmark

In [27]:
items = [
    "Чайные ложки",
    "Ремень",
    "Лопатки (текстолит) на вакуумный насос КО-503Б (комп)",
    "Отвод 73",
    "Зарядное устройства Motorola NNTN8117",
    "Оборудование для лаборатории анализа нефти и воды",
    "Лента клейкая 18мм*10м",
    "Оксолиновая мазь 10г 0,25%",
    "Щебень",
    "Бумажные полотенце",
    "Перчатки диэлектрические",
    "Системный блок",
    "Втулка подшипника Н 13,3,281,01,016",
    "Масляный фильтр ЯМЗ-238",
    "Дрель-шуруповёрт DeWALT DCD776S2",
    "Муфта сцепления в сборе ЗИЛ-131",
    "Клапан электромагнитный ВН2Н-6Е ФЛ.УХЛ1 - В СБОРЕ. Исполнение: фланцевое, DN50 мм, 6 бар, 220В, 50 Гц. для печей ПБТ-1,6М",
    "Муфта д.50мм, пластик",
    "Уровнемер для МДУ, Kenco Tulsa",
    "Автоматический выключатель ВА57-400 3Р 400А",
    "Датчик скорости Волга, Газель, Соболь, УАЗ",
    "Кислота паяльная 30мл",
    "Глушитель УАЗ 330395.330394.390945 Евро-2,3,4",
    "Фильтр грубый очистки 974-897",
    "Манометр 0-250 кгс/см2 (М 20*1,5)",
    "Шток полированный 1 1/4-8000-7/8-45",
    "Огнетушитель ОУ-5",
    "Ацетилсалициловая кислота 0,5 №10",
    "Редуктор газовый",
    "Труба ПВХ",
    "Фильтр семник",
    "Ткань для уборки",
    "Паранит 3 мм",
    "Точка доступа Ubiquiti UniFi 6+",
    "Разъединитель РЛНД 10кв",
    "Растворитель 646 10л",
    "КЛЮЧ газовый №0 СИБРТЕХ",
    "Электрод LB -2,6",
    "Дополнительный насос отопителя 12Вт",
    "Лист металлический 5мм",
    "Труба д. 20мм, пластик",
    "Агрегат ГА-165",
    "Спецодежда (костюм) зимняя для сварщика",
    "Фланец 80/25 ГОСТ 12820-80",
    "Дехлор (банк -300 таб)",
    "Долото 105мм",
    "Аккумулятор 75",
    "Кран шаровый Ду 80 Стал, Фланц.",
    "Ремень привода компрессора 16х11-1103",
    "Серверные жесткие диски",
    "Ластик",
    "Наконечник 35",
    "Костюм форменный для военнослужащих (Спецодежда летняя для охраны)",
    "Стартер",
    "Белизна (банк -500 гр.)",
    "Сальник 42х75х10 правый 10 шт",
    "Аппарат топливный ТНВД МТЗ -80",
    "Штанга короткая насосная 22-2,0-С",
    "ТНВД топливной аппаратуры КАМАЗ ЕВРО-2",
    "Тиски слесарные",
    "Воздушный фильтр КАМАЗ ЕВРО 1 7405Р,1109560",
    "Нижняя щека и штифт в сборе для трубного ключа 48\"",
    "Автоматический выключатель 3-х фазный 32 А",
    "Крышка сальника насоса К100-65-250а-С",
    "Гибкий шланг сместителя - 60 см.",
    "Ручка шариковая синий",
    "Автоматический выключатель 3-х фазный 250А",
    "Сахар 1кг рафинад",
    "Пускатель магнитный 63А",
    "Фильтр масляный 65.05510-5020В на ДЭС-825 АКСА",
    "Пакеты большие",
    "Печка радиатор салона",
    "Сканер штрих кодов Honeywell Orbit 7120",
    "Краска Албан 3,5 синий",
    "Фильтр топливный КАМАЗ",
    "Канат ГОСТ 7665-80 д.14.5мм(Г-В-Н-Р-Т-1770)",
    "Аккумулятор пожарный SF12045 12V 4,5A",
    "Барабан WC7120/7220 (67000k) чёрный (R1)",
    "Пластырь перцовый",
    "Клей Момент водостойкий125мл",
    "Щетка генератора с держателем 27370-58460",
    "Литол-24/О/смазка/18кг/шт",
    "Ручка синяя",
    "Клапан обратный Dn-89мм Pn-16bar",
    "Чай черный 100 пак",
    "Набор ключей от 32 до 55мм (хромованадиевые)",
    "Фильтр топливный 65.12503-5018А на ДЭС-825 АКСА",
    "Отрезной диск 125 мм",
    "Жидкость омыватель Sibiria 4л",
    "Тонер Xerox (малиновый) VersaLink C7020",
    "Альбуцид 30% 10мл гл.капли",
    "Кран шаровой д. 50мм, пластик",
    "Пускатель магнитный 32А",
    "Закуп Печи подогрева по объекту ГУ-2 на м/р Сарыбулак",
    "Ремень помпы",
    "Электрический подогреватель охлаждающей жидкости",
    "Манометр",
    "Крышка КПП с КОМ",
    "АЛЛЕРГОНАФ 15 МЛ КАПЛИ Д/ГЛАЗ И НОСА",
    "Кабель ВБбШВ 4*35",
    "Замок висячий",
    "Валокардин 20 мл",
    "Тройник д.50мм, пластик",
    "Хоз. Мыло",
    "Дизельное масло",
    "Сварочный аппарат САГ c дизельным двигателем Линкольн",
    "Набор инструментов FORCE 142 предм.",
    "Штанга короткая насосная 22-1,0-С",
    "Фильтр масляный Краз",
    "Ремень генератора 90916-02452",
    "Датчик масленый",
    "Медные Жала для Паяльника 6мм",
    "Лопата штыковая",
    "Рулетка 20 м",
    "Сверло набор 1мм-10мм",
    "Тормозная лента ГКШ-1500",
    "Лимон",
    "Распределительный шкаф ПР-11",
    "Сапоги резиновые с чулком",
    "Санипласт № 10",
    "Лента клейкая 48 мм*132м",
    "АФНИ.306121.005 Кран шаровый Ду 50 Ру70",
    "Прямой трубный ключ, № Модели 24, типоразмер 24\"/600мм, размер трубы 3\"/80мм, вес 4,4кг",
    "Бура для пайки",
    "Фильтр салона CF 3504C",
    "Софрадекс 5мл",
    "Амортизатор передний",
    "Только секции насоса ВНН5-80-1200",
    "Тройник д.40мм, пластик",
    "Йокс спрей 30 мл",
    "Переключатель поворота",
    "Стропа текстильная петлевая СТП8-3000",
    "Сальник передний на ДЭС-825 АКСА (турецкий)",
    "Олово для пайки",
    "Радиатор охлаждения ЗИЛ",
    "Дрель электрический",
    "Ключ ударный (плюха) 46",
    "Grandstream UCM6308A - IP ATC",
    "Нож столовый",
    "Аккумулятор 12V-7A SF",
    "Американка д.63мм, пластик",
    "Фужеры для воды(сока)",
    "Мост",
    "Флюс для пайки",
    "Эргоферон №20",
    "Масляная станция МС-1",
    "Масляный фильтр",
    "Кресло для ИТР (офис Сарыбулак)",
    "Терморегулятор для электрического обогревателя на 300 градусов",
    "Труба 114*5",
    "Мышь USB, проводной",
    "Шарики для унитаза",
    "Фенистил гель30г",
    "Зажим для груза парафинорезки",
    "Крестовина кардана Волга, Газель, УАЗ \"СТАНДАРТ\"",
    "Аккумулятор УПС NPW36-12",
    "Гайка М16",
    "Ежедневник",
    "Фланец Dn-50мм Pn-16bar",
    "Хомут У-1",
    "Манометр 0-1 кгс/см2",
    "Штанга короткая насосная 22-1,5-С",
    "Шнур асбестовый 10мм",
    "Метрошток 4,5 м",
    "Сальбутамол аэрозоль 200доз",
    "Электробензонасос",
    "Кран шаровой д.15мм, пластик",
    "Аркан джутовая удерживающая 18 мм",
    "Задвижка шаровая Dn-100мм Pn-16bar",
    "Рубильник 630 Ампер",
    "Насос ДАВ А50/180 ХМ",
    "Отвод 50",
    "Плашки 73 под спг 75",
    "Двутавр 20",
    "Труборез RIDGIT 6-S 114-168 мм",
    "Инструмент для развальцовки(разбартовки) медных трубок",
    "Ремень генератора 10000-18860",
    "Регулятор РДБК 1-50/35",
    "Конвертор универсальный GM-211",
    "Кабель micro USB",
    "Гайка М 6",
    "Масло И40",
    "Набор сверл по металлу 25 шт",
    "Клавиатура, проводная",
    "Корпус насоса К100-65-250а-С",
    "Лампа NL-T8 LED 18 W 1200 мм",
    "Омез 20мг",
    "Ножницы по металлу",
    "Вал карданный УАЗ",
    "Штанга насосная 22-8000мм-Д с 3 центраторами",
    "Насос ВНН5-50-1500/53-004921-2",
    "Воздушный компрессор",
    "Промежуточное звено ПРТ",
    "Штанга короткая насосная 22-0,5-С",
    "Вентиль шаровый д.25мм, бронза",
    "Вывоз, переработка и утилизация отходов потребления",
    "Труба б/ш 89х6",
    "Гриппостад вит С № 10",
    "Клапан обратный Dn-219мм Pn-16bar",
    "Фильтр масляный 901-115",
    "Насос 25-175RHAM-14-5-2-2",
    "Шланг в/дDy=6мм(2SN),(L=30м)",
    "Ингавирин 90 мг №7",
    "Натрия тиосульфат-Дарница 30% 5мл №10",
    "Литол-24 10 кг",
    "Кран шаровой д.20мм, пластик",
    "Салфетки Клинекс",
    "Тормозной суппорт",
    "Набор ключей 6-32",
    "Распределительная коробка JBM-100 EP",
    "Хомут пластиковый 3,6х5 (100шт в упаковке)",
    "Кетонал свечи №12",
    "Насос бочковой ручной для емкостей",
    "Лента сигнальная Осторожно кабель",
    "Резак по металлу",
    "Тетрациклин АКОС 3%-15г мазь",
    "Рулевая тяга",
    "Ремень А-850",
    "Фланец 200",
    "Лента ФУМ-1 высший сорт 0,10х60 ТУ 6-05-1388-86",
    "Экстракт валерьянки 20мг № 50",
    "Насос дозатора с шлангой DLX MA/AD",
    "Вентиль шаровый д.50 мм, бронза",
    "Закуп Дренажная емкость по объекту ГУ на м/р Кайнар",
    "Порошок Пропер",
    "Сальник лобовой",
    "Палец сбивной для сливного клапана КС-73",
    "Электро насос бочковой взрывозащищенный",
    "Автоматический выключатель 3-х фазный 100А",
    "Крышка корпуса насоса К100-65-250а-С",
    "Пропан",
    "Концевая заделка Е-19",
    "Лампа паяльная",
    "Lan-тестер кабельный щумозащищенный ProsKit MT-7029",
    "Антенна автомобильная Diamond MC201 340-520 МГц, 1/4, 70Вт, с магнитным креплением и кабелем 4м, BNC",
    "Принт-картридж Xerox B1025 (80K)",
    "Валидол 0,06 № 10",
    "Масло трансформаторное Т-1500",
    "Болт М12",
    "Азот",
    "Датчик-реле контроля пламени СЛ-90-1/24Е",
    "Обустройство устья скважин 222,218,219,317,318 на м/р Сарыбулак",
    "Пена монтажная",
    "Соль пищевая",
    "Парацетамол 0,5г",
    "Шланг кислородный ф6мм",
    "Набор инструментов 142 предмет",
    "Комплект досмотровых средств ПОИСК-2У",
    "Изолента ХБ тканевая черная",
    "Амортизатор задний масляный",
    "Краскопульт",
    "Герметик",
    "Растворитель 646",
    "Паста водочувствительная",
    "Кабельный наконечник на груза",
    "Капсикам мазь 30",
    "Уголь активированный № 10",
    "Картридж на принтер Canon 4410",
    "Свечи зажигания (к-т: 4шт)",
    "Ремень ГРМ Renault",
    "ГКШ, сухари 73мм",
    "Железная терка",
    "Ремень 997-871",
    "Главный тормозной цилиндр",
    "Задвижка фл.ст. Ду80 Ру16",
    "Аккумулятор 90",
    "УВН-10кВ световая индикация и сигнальная",
    "Обратный клапан PCV",
    "Перчатки рабочие х/б с ПВХ",
    "Герметик Автосил 180 гр",
    "Фильтр влагоотделитель 4040.0216",
    "Скорошиватель бумаж.",
    "Барабан WC7120/7220 (51000k) пурпурный (R3)",
    "Кабель ВБбШВ 4х16мм",
    "Нефтегазовый сепаратор НГС-II-1.8-1200",
    "Муфта д.25мм, пластик",
    "Скважинный нагреватель НСВ-4 в сборе",
    "Краска синяя",
    "Лента транспортерная (двухниточная)",
    "Фастум гель 50г 2,5%",
    "Корректор-20мл (RETYPE)",
    "Масло полифитовое 100мл",
    "Балка проблесковая оранжевая светодиодная четырехсторонняя 180 Ватт",
    "Опора кардана (подвеска)",
    "Моторчик отопителя салона",
    "Степлер №10",
    "Дисплей расходомера Endress & Hause PROWIRL F",
    "Выключатель автоматический 50 А",
    "Хоз.мыло",
    "Шланг кислородный",
    "Полотно для ножовки по металлу",
    "Конфеты ассорти",
    "Ципролет 500мг № 10",
    "Прожектор LED 200 W",
    "Губка",
    "Витамин В6-5% № 10",
    "Шлифовочный диск 180 мм",
    "Лоток перфорированный 100х80х2500",
    "Гайка М16 (новый)",
    "Wi-Fi роутер, Beeline",
    "Обогреватель",
    "Очки защитные темные",
    "Масло моторное Лукойл Люкс 10/40",
    "Подставка под РЛНД с приводом",
    "Сальник 42х68х10",
    "Мильгамма 2 мл №10",
    "Синафлан мазь",
    "Фильтр сепаратора 4010.0052",
    "Хомут Х-1",
    "Мин.вода Тассай 0,5 л",
    "Кондиционер настенный Almacom 18 QS",
    "Мультиметр цифровой ANTNG A3005",
    "Фильтр воздушный 400504-00159 на ДЭС-825 АКСА",
    "Подшипник 2217-3104800 заднего колеса (сальник65х90х10)",
    "Щетка металлическая с деревянной ручкой",
    "Верхняя щека для трубного ключа 24\"",
    "Барабан VersaLink C7020/25/30 (черный - 109000к, цветные - 87000к)",
    "Фонарь ручной",
    "Анаферон взрослый",
    "Термокабель 15KXTV2-CT-T3",
    "Сальниковая набивка 8 мм",
    "Шуруп 3,5*25",
    "Фильтр масляный 996-452 (10000-59645, CH10929)",
    "НАЙЗ 1% ГЕЛЬ 20 г",
    "Набор медных шайб",
    "Кетатоп 100мг 2,0 №10",
    "Хомуты 25-40 мм",
    "ВНКТ 73-5,5 и муфты к ним",
    "Амортизатор задний",
    "Газосепаратор-диспергатор ГСДАШО5-200Э/КП",
    "Задвижка клиновая Dn-50мм Pn-16bar",
    "Трубный ключ большой",
    "Блок управление двиг. в сборе",
    "Бак для приготовления гипохлорида -60л",
    "Наконечник 16",
    "Пульт - приемник S2400ND",
    "Головка сальниковая для лубрикатора",
    "Помпа водяной",
    "Манометр 0-10 кгс/см2 (М 20*1,5)",
    "Канат страховочный 16мм",
    "Радиатор кондиционера",
    "Лопата совковая",
    "Матрас",
    "Ножницы по металлу SPARLUX",
    "Американка д.20мм, пластик",
    "Видеорегистратор Neoline Wide S22",
    "Дисковая пила",
    "Насос дозировочный УДХ",
    "Блок индикации и регистрации параметров СПО СПС-7Т",
    "Ваза для цветов",
    "Вентиль шаровый д.15мм, бронза",
    "Грипго № 4",
    "Шкаф коммуникационный, SHIP, 601S.6624.03.100, 103 серия, 19'' 24U, 600*600*1200 мм, Ш*Г*В, Передняя дверь закалённое стекло, Задняя дверь глухая",
    "Предохранитель 15А для радистанции Motorola CM 140",
    "Термодатчик МЕТРАН-2000 E07 Исполнение У1.1 Диапазон -50...+120 L28, HCX Pt100/B/4 для узла подвода УПН-5-16-1800 горизонтального насоса",
    "Насос погружной подача сточных вод на фильтрацию PEDROILOV",
    "Меновазин 40",
    "Порошок (мешок - 15 кг.)",
    "Набор головок",
    "Удобрение для цветов",
    "Деэмульгатор",
    "Фильтр масляный",
    "Банат с ведром",
    "Тройник д.25мм, пластик",
    "Насос (К-45/30 Уз1)",
    "Сетевой UTP - кабель, CAT.5E",
    "Превентор плашечный ручной ППР 60х21",
    "Краска желтая по металлу",
    "Торцевой фрез 146мм",
    "Муфта д.40мм, пластик",
    "Ролик ремня генератора Toyota Hilux",
    "Колодка тормозная",
    "Мыло жидкое",
    "Сифон гофрированный",
    "ОЗУ 16 Гбайт DDR4",
    "Вентиль шаровый д.20мм, бронза",
    "Реактивная тяга на сайлентблоках ВАЗ",
    "Набор инструментов 94 предмет",
    "Бинокль 15*50",
    "Марля не/ст 1*90",
    "Кабель КГ2х2,5",
    "Туалетное мыло (жидкое мыло 5-л.)",
    "Болт М 6",
    "AEL пакетный выключатель",
    "Диклофенак 75мг",
    "Ручка красная",
    "Комет",
    "Очиститель карбюратора",
    "Режущий ролик RIDGIT F367S",
    "Плита дорожная 6 х 1,5",
    "Набор диэлектрического инструмента ЗУБР \"ЭЛЕКТРИК\" пассатижи бокорезы клещи переставные кабелерез, разводной ключ",
    "Фланец стальной 100/25",
    "Удлинитель УК50 Катушка 4места/50м",
    "Электро-контактный термометр ТГП-100Эк 0-100°С L=5м",
    "Кабель USB на манометров PPS-25",
    "Масло В.О. ТАД-17 10 л",
    "Ремень 8,5*8*850",
    "Фонарь налобный 18650",
    "Линкас мазь",
    "Задвижка клиновая Dn-89мм Pn-16bar",
    "Рем комплект двигателя прокладок 4216",
    "Стекло лобовое",
    "Диклофенак 140мг № 5, пластырь",
    "Компрессор кондиционера",
    "Комбинезон одноразовый",
    "Домкрат 3тн",
    "Точка доступа Ubiquiti Rocket M5 AC Lite",
    "Мукалтин 0,05",
    "Тройник д.15мм, пластик",
    "Тонер Xerox (голубой) VersaLink C7120",
    "Лоток перфорированный 100х50х2500",
    "Аккумулятор газоанализатора 12V 12AR",
    "Монтажный нож электрика",
    "Обратный клапан ТНВД",
    "Костюм влагозащитный",
    "Колено 90º (муфта, угольник проходной)",
    "Системный блок Core i5-12400-2.5GHz/H610/RAM 16GB/SSD 1TB (M.2)/no DVD/400W",
    "Кран шаровой д.32мм, пластик",
    "Ингибитор солеотложений \"Ранскейл-4107\"",
    "Файл в наборе А4",
    "Замок двери правый 315177-6105100-00 Уаз Хантер",
    "Рулевой механизм УАЗ",
    "Генератор 27060-ОС110",
    "АЛМАГЕЛЬ 170,0",
    "Малярный валик",
    "Набор ключей от 8 до 30(хромованадиевые)",
    "Краска серая (расход-200 гр/м.кв.)",
    "Кран шаровой д.40мм, пластик",
    "Колодка тормозная передняя Газель (комплект)",
    "Автоматический выключатель 3-х фазный 40А",
    "Мышь беспроводная",
    "Литол-24/О/смазка/15кг/шт",
    "Шаровая опора 43310-09015 (нижний)",
    "Фонарь налобный",
    "Ремень 1320",
    "Шланги системы охлаждения на ДЭС ВИЛСОН",
    "Клей Akfix-705 400 ml+100gr",
    "Набор электрика",
    "Держак",
    "Отрезной диск 230 мм",
    "Авиационное масло",
    "Переносной светодиодный прожектор",
    "Колодки тормозные (FR) Hilux передние",
    "Верхняя щека для трубного ключа 36\"",
    "Маркер перманентные в наборе",
    "Лист ПВЛ 5(506) 1200х3000",
    "Швеллер10 У 12м",
    "Переключатель массы",
    "Масляный фильтр Д-245 ФМ 009-1012005 (Т227)",
    "Амортизатор передний УАЗ-3163",
    "Манометр 0-25 кгс/см2 (М 20*1,5)",
    "Псило бальзам 20г",
    "Электрод LB -3,2",
    "Аква - Алкалин, щелочный реагент",
    "Коммутатор Ubiquiti UniFi Switch Pro 48 PoE Gen2",
    "Болт М16",
    "Масло моторное Лукойл Авангард 10W-40 205л",
    "Левомицетин 0,5% 10 мл капли глаз",
    "Литиевая батарея от PPS-25",
    "SSD - накопитель, 1 Tb, 2.5\"",
    "Патч Панель, SHIP, P197-24C, Цветная, Категория 5e, 19\" (1U), 24 Порта",
    "Рулевой тяга (наконечники) 45046-09281",
    "Бензин АИ-92 л",
    "Набор вольцовка",
    "Патрубка радиатора на ВИЛСОН",
    "Спиртовые салфетки",
    "Преобразователь полированного штока 3071 Эхолот Ecometer",
    "Бур 8*160 мм",
    "ДЮФАЛАК 200 МЛ Р-Р",
    "Прожектор LED 100 W",
    "Бумага А 4",
    "Топливный фильтр",
    "Сверло по металлу набор",
    "Полотенце",
    "Сальник задний на ДЭС-825 АКСА (турецкий)",
    "Журнал регистрации \"Исходящей корреспонденции\"",
    "Чехлы для Автомобильных сидений",
    "Топливный фильтр 186100-6360",
    "Паронит 5 мм",
    "Воздушный фильтр TOYOTA 17801-61030",
    "Коврик термоизоляционный",
    "Сайлентблок продольной тяги",
    "Муфта концевая 10 КВТп-3х(150-250)",
    "Гидроциклон для очистки воды от мех.примесей печей ПП-0,63 на ГУ-1",
    "Комплект ГРМ",
    "Кран шаровой стальной привар. Ду 100/25",
    "Фильтр газовый Термобрест ВН6-6М",
    "Набор метчиков и плашек 000110М18,1,5мм,110шт комп",
    "Душевая кабина",
    "Тормозной колодка на барабан УПА-60/80.размер 20*12см",
    "Отвод 114*5",
    "Вакуумный насос КО503Б,02,14,100-Q=4М3/мин=1450об/мин",
    "Датчик давления масло на ДЭС-825 АКСА",
    "Только секции насоса 0215ЭЦНАКИ5-25И-1550Э/КП",
    "Календарь настольный",
    "Кабель HDMI",
    "Силикон-сапфировый автономный манометр PPS25",
    "Амбробене 30мг № 20",
    "Термометр WIKA 0 - 100 ºC, L = 50 mm ТМ 55.01",
    "Форсунка ЯМЗ"
]

In [29]:
# Создаем DataFrame для хранения результатов
data = []

# Обрабатываем каждый элемент из списка items
for item in items:
    # Получаем предсказания от модели
    predictions = pipeline.search(item, top_k=3)['results']
    
    # Формируем строки для Prediction@3
    pred3_lines = []
    for i, pred in enumerate(predictions[:3]):  # Берем топ-3 предсказания
        code = pred['metadata']['code'] if 'metadata' in pred and 'code' in pred['metadata'] else ''
        text = pred['text'] if 'text' in pred else ''
        pred3_lines.append(f"{code}: {text}")
    
    # Добавляем запись в данные для DataFrame
    data.append({
        "Наименование": item,
        "Prediction@1": predictions[0]['metadata']['code'] if len(predictions) > 0 else '',
        "Prediction@3": "\n".join(pred3_lines) if pred3_lines else ''
    })

# Создаем DataFrame
df = pd.DataFrame(data)

100%|██████████| 1/1 [00:00<00:00, 78.14it/s]


In [30]:
df

,Наименование,Prediction@1,Prediction@3
0,Чайные ложки,108313.200.000001,"108313.200.000001: Чай черный, гранулированный..."
1,Ремень,221940.300.000000,"221940.300.000000: Ремень клиновый, приводный\..."
2,Лопатки (текстолит) на вакуумный насос КО-503Б...,239914.000.000019,239914.000.000019: Комплект уплотнительных кол...
3,Отвод 73,245130.300.000000,"245130.300.000000: Отвод чугунный, диаметр 50 ..."
4,Зарядное устройства Motorola NNTN8117,265152.350.000003,265152.350.000003: Расходомер электромагнитный...
...,...,...,...
495,Кабель HDMI,273213.500.000004,273213.500.000004: Кабель специализированный т...
496,Силикон-сапфировый автономный манометр PPS25,265152.750.000002,265152.750.000002: Микроманометр многопредельн...
497,Амбробене 30мг № 20,234411.000.000062,234411.000.000062: Воронка Бюхнера № 3\n234411...
498,"Термометр WIKA 0 - 100 ºC, L = 50 mm ТМ 55.01",265151.300.000001,265151.300.000001: Термометр ТСМ-2-1 0-100С\n2...


In [31]:
# Сохраняем в Excel (опционально)
df.to_excel("predictions_results.xlsx", index=False)